In [ ]:
# load and plot dataset
import pandas as pd

In [ ]:
train_df = pd.read_csv("train_preprocess_1.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
test_df['day']=test_df['event_datetime'].str.slice(8,10)
test_df['dayofweek']=test_df['day'].map({'01':'2',
                                           '02':'3',
                                           '03':'4',
                                           '04':'5',
                                           '05':'6',
                                           '06':'0',
                                           '07':'1',
                                           '08':'2',
                                           '09':'3',
                                           '10':'4',
                                           '11':'5',
                                           '12':'6'})

test_df['hour'] = test_df['event_datetime'].str.slice(11,13)
data=test_df.loc[:,['bid_id', 'hour']]
hour0=data[data['hour']=='00']
hour1=data[data['hour']=='01']
hour2=data[data['hour']=='02']
hour3=data[data['hour']=='03']
hour4=data[data['hour']=='04']
hour5=data[data['hour']=='05']
hour6=data[data['hour']=='06']
hour7=data[data['hour']=='07']
hour8=data[data['hour']=='08']
hour9=data[data['hour']=='09']
hour10=data[data['hour']=='10']
hour11=data[data['hour']=='11']
hour12=data[data['hour']=='12']
hour13=data[data['hour']=='13']
hour14=data[data['hour']=='14']
hour15=data[data['hour']=='15']
hour16=data[data['hour']=='16']
hour17=data[data['hour']=='17']
hour18=data[data['hour']=='18']
hour19=data[data['hour']=='19']
hour20=data[data['hour']=='20']
hour21=data[data['hour']=='21']
hour22=data[data['hour']=='22']
hour23=data[data['hour']=='23']

del data

hour0['click_per_hour']=0.088928
hour1['click_per_hour']=0.080612   
hour2['click_per_hour']=0.082022   
hour3['click_per_hour']=0.090319  
hour4['click_per_hour']=0.101889   
hour5['click_per_hour']= 0.098369   
hour6['click_per_hour']=0.098494
hour7['click_per_hour']=0.09979   
hour8['click_per_hour']=0.100691
hour9['click_per_hour']=0.108933   
hour10['click_per_hour']=0.101265  
hour11['click_per_hour']= 0.097247   
hour12['click_per_hour']=0.097044   
hour13['click_per_hour']=0.098666
hour14['click_per_hour']=0.103007 
hour15['click_per_hour']=0.065717   
hour16['click_per_hour']=0.064247   
hour17['click_per_hour']=0.063465   
hour18['click_per_hour']=0.058997   
hour19['click_per_hour']=0.062006   
hour20['click_per_hour']=0.070008   
hour21['click_per_hour']=0.076275   
hour22['click_per_hour']=0.094003   
hour23['click_per_hour']= 0.092863

click_per_hour=pd.concat([hour0,hour1,hour2,hour3,hour4,hour5,hour6,hour7,hour8,hour9,hour10,hour11,hour12,hour13,hour14,
                         hour15,hour16,hour17,hour18,hour19,hour20,hour21,hour22,hour23],axis=0)
click_per_hour=click_per_hour.drop(['hour'],axis=1)
click_per_hour=click_per_hour.reset_index(drop=True)

test_df=test_df.merge(click_per_hour,on='bid_id')
test_df

test_df=test_df.drop(['event_datetime','day'],axis=1)

In [ ]:
fs_col = ['campaign_id','media_domain','ssp_id','device_os_version','device_language','device_region','advertisement_id','device_connection_type','adset_id','device_carrier','device_city','device_make','device_model']

In [ ]:
def get_complement(train, test, i):
  a = set(pd.get_dummies(train[i+'_fs']).columns)
  dummied = pd.get_dummies(test[i])
  b = set(dummied.columns)
  dummied.drop(list(b-a), axis=1, inplace=True)
  return dummied
  

In [ ]:
def output(i, dum_train):
  name = 'others_'+i
  name2 = i+'_fs'
  dum_train = dum_train*2
  dum_train[name] = 1
  train_out = pd.DataFrame(dum_train.idxmax(axis=1), columns=[name2])
  return train_out

In [ ]:
test_set = test_df['bid_id']
for i in fs_col:
  dummy_test = get_complement(train_df, test_df, i)
  test_out = output(i, dummy_test)
  del dummy_test
  test_set = pd.concat([test_set, test_out], axis=1, ignore_index=True)
  del test_out

test_set.columns = ['bid_id']+fs_col

In [ ]:
test_set.columns=['bid_id', 'campaign_id_fs', 'media_domain_fs', 'ssp_id_fs', 'device_os_version_fs',
       'device_language_fs', 'device_region_fs', 'advertisement_id_fs',
       'device_connection_type_fs', 'adset_id_fs', 'device_carrier_fs', 'device_city_fs',
       'device_make_fs', 'device_model_fs']
del train_df

In [ ]:
test_df=test_df.drop(fs_col,axis=1)
test_df=test_df.merge(test_set,on='bid_id')
test_df=test_df.drop(['media_bundle','device_country','media_name'],axis=1)

In [ ]:
cate=pd.read_csv('audience_preprocess_2.csv')

In [ ]:
#모델 불러오기

from deepctr.layers import custom_objects
from tensorflow.python.keras.models import  save_model,load_model
model = load_model('xDeepFM_FS_full_128_10.h5',custom_objects)# load_model,just add a parameter

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
test_df = test_df.merge(cate, how='left', on="device_ifa")
del cate

In [ ]:
na_index = test_df.loc[pd.isna(test_df["cate_answer_1"]), :].index

na_index = test_df.loc[pd.isna(test_df["cate_answer_1"]), :].index
test_df.loc[test_df.index[na_index], 'cate_answer_1']=22.2131
test_df.loc[test_df.index[na_index], 'cate_answer_2']= 49.3176
test_df.loc[test_df.index[na_index], 'cate_answer_3']= 15.9415
test_df.loc[test_df.index[na_index], 'cate_answer_4']=  51.3909
test_df.loc[test_df.index[na_index], 'cate_answer_5']=  54.7647
test_df.loc[test_df.index[na_index], 'cate_like_1']= 13.1013
test_df.loc[test_df.index[na_index], 'cate_like_2']=5.61905
test_df.loc[test_df.index[na_index], 'cate_like_3']=9.20284   
test_df.loc[test_df.index[na_index], 'cate_like_4']=6.52821   
test_df.loc[test_df.index[na_index], 'cate_like_5']= 8.06916

test_df[['age']]=test_df[['age']].fillna('old')
test_df[['gender']]=test_df[['gender']].fillna('U')
test_df[['marry']]=test_df[['marry']].fillna('C')

###XDeepFM Prediction

In [ ]:
sparse_features = ['ssp_id_fs', 'campaign_id_fs', 'media_domain_fs',
       'device_os_version_fs', 'device_language_fs', 'device_region_fs',
       'advertisement_id_fs', 'device_connection_type_fs', 'adset_id_fs',
       'device_carrier_fs', 'device_city_fs', 'device_make_fs',
       'device_model_fs', 'device_ifa', 'dayofweek', 'hour','media_id','placement_type','publisher_id','publisher_name','device_os','age','gender','marry'] 
test_df[sparse_features] = test_df[sparse_features].astype(str)

In [ ]:
sparse_features = ['device_ifa', 'dayofweek', 'hour', 'media_id',
    'placement_type', 'publisher_id', 'publisher_name', 'device_os',
    'ssp_id_fs', 'campaign_id_fs', 'media_domain_fs',
    'device_os_version_fs', 'device_language_fs', 'device_region_fs',
    'advertisement_id_fs', 'device_connection_type_fs', 'adset_id_fs',
    'device_carrier_fs', 'device_city_fs', 'device_make_fs',
    'device_model_fs', 'gender', 'age', 'marry'] 
dense_features = [ 'cate_answer_1', 'cate_answer_2', 'cate_answer_3', 'cate_answer_4',
    'cate_answer_5', 'cate_like_1', 'cate_like_2', 'cate_like_3',
    'cate_like_4', 'cate_like_5']

test_df[sparse_features] = test_df[sparse_features].fillna('-1', )
test_df[dense_features] = test_df[dense_features].fillna(0, )

In [ ]:
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import *
from deepctr.inputs import  SparseFeat, DenseFeat, get_feature_names

#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
import numpy as np

In [ ]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=test_df[feat].nunique(),embedding_dim=4, use_hash=True, dtype='string')
                        for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                        for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
holdout_model_input = {name:test_df[name] for name in feature_names}

In [ ]:
#batch_size 조절!
submit = model.predict(holdout_model_input, batch_size=128)

In [ ]:
#모델명이랑 동일하게 저장 부탁드려요
submit = pd.DataFrame(submit)
submit = pd.concat([test_df['bid_id'], submit], axis=1)
submit

,bid_id,0
0,jILrN0gGpx,0.267120
1,zA3WyymOcJ,0.036325
2,PwIj11RYvM,0.033930
3,W0o0KwmTSQ,0.024854
4,UpL3kLWqZy,0.033757
...,...,...
549995,bSxh3i0gN3,0.214208
549996,LAyxamwNxm,0.190432
549997,3sF8PXgPom,0.353014
549998,W8XuFXZw4v,0.178251


In [ ]:
submit.to_csv("test_predict.csv",index=False, header=False)